In [1]:
import os
import sys
import pyspark
from pyspark.sql import SparkSession

# 172.21.0.2: docker inspect -f '{{range.NetworkSettings.Networks}}{{.IPAddress}}{{end}}' minio
spark = SparkSession.\
        builder.\
        appName("My Application"). \
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        config("spark.hadoop.fs.s3a.access.key", f"{os.environ['MINIO_ROOT_USER']}").\
        config("spark.hadoop.fs.s3a.secret.key", f"{os.environ['MINIO_ROOT_PASSWORD']}").\
        config("spark.hadoop.fs.s3a.endpoint", f"http://{os.environ['MINIO_IP']}:9000").\
        config("spark.eventLog.enabled", "true").\
        config("spark.eventLog.dir", "s3a://spark-events/").\
        config("spark.history.fs.logDirectory", "s3a://spark-events/").\
        config("spark.hadoop.fs.AbstractFileSystem.wasb.Impl", "org.apache.hadoop.fs.azure.Wasb").\
        config(f"spark.hadoop.fs.azure.account.key.{os.environ['AZURE_STORAGE_ACCOUNT']}.blob.core.windows.net", f"{os.environ['AZURE_ACCESS_KEY']}").\
        config("spark.hadoop.fs.azure.block.blob.with.compaction.dir", "/hbase/WALs,/data/myblobfiles").\
        config("spark.hadoop.fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem").\
        config("spark.hadoop.fs.azure.enable.append.support", "true").\
        getOrCreate()

2023-12-17 09:35:00,520 INFO spark.SparkContext: Running Spark version 3.3.0
2023-12-17 09:35:00,672 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-12-17 09:35:00,769 INFO resource.ResourceUtils: ==============================================================
2023-12-17 09:35:00,769 INFO resource.ResourceUtils: No custom resources configured for spark.driver.
2023-12-17 09:35:00,770 INFO resource.ResourceUtils: ==============================================================
2023-12-17 09:35:00,771 INFO spark.SparkContext: Submitted application: My Application
2023-12-17 09:35:00,804 INFO resource.ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 512, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
2023-12-17 09:3

In [2]:
conf = spark.sparkContext.getConf().getAll()

# Print Configuration
for k,v in conf:
    print(f"{k} : {v}")

spark.hadoop.fs.azure.block.blob.with.compaction.dir : /hbase/WALs,/data/myblobfiles
spark.executor.memory : 512m
spark.eventLog.enabled : true
spark.app.startTime : 1702805700506
spark.eventLog.dir : s3a://spark-events/
spark.driver.extraJavaOptions : -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED
spark.app.submitTime 

In [3]:
df = spark.read.csv(f"wasbs://{os.environ['AZURE_STORAGE_CONTAINER']}@{os.environ['AZURE_STORAGE_ACCOUNT']}.blob.core.windows.net/yellow_tripdata_2021-01.csv", header=True)

2023-12-17 09:38:32,752 INFO internal.SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
2023-12-17 09:38:32,756 INFO internal.SharedState: Warehouse path is 'file:/opt/workspace/code/spark-warehouse'.
2023-12-17 09:38:32,776 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@247b1c5b{/SQL,null,AVAILABLE,@Spark}
2023-12-17 09:38:32,777 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@50ecd47b{/SQL/json,null,AVAILABLE,@Spark}
2023-12-17 09:38:32,778 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@292a0769{/SQL/execution,null,AVAILABLE,@Spark}
2023-12-17 09:38:32,779 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@7a0ee75d{/SQL/execution/json,null,AVAILABLE,@Spark}
2023-12-17 09:38:32,781 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@2191bcec{/static/sql,null,AVAILABLE,@Spark}
2023-12-17 09:38:33,688 INFO impl.MetricsConfig: Loaded properties from 

In [4]:
df.show()

2023-12-17 09:39:23,366 INFO datasources.FileSourceStrategy: Pushed Filters: 
2023-12-17 09:39:23,367 INFO datasources.FileSourceStrategy: Post-Scan Filters: 
2023-12-17 09:39:23,367 INFO datasources.FileSourceStrategy: Output Data Schema: struct<VendorID: string, tpep_pickup_datetime: string, tpep_dropoff_datetime: string, passenger_count: string, trip_distance: string ... 16 more fields>
2023-12-17 09:39:23,389 INFO memory.MemoryStore: Block broadcast_3 stored as values in memory (estimated size 304.5 KiB, free 433.4 MiB)
2023-12-17 09:39:23,400 INFO memory.MemoryStore: Block broadcast_3_piece0 stored as bytes in memory (estimated size 54.0 KiB, free 433.3 MiB)
2023-12-17 09:39:23,401 INFO storage.BlockManagerInfo: Added broadcast_3_piece0 in memory on a31eab60218a:43207 (size: 54.0 KiB, free: 434.2 MiB)
2023-12-17 09:39:23,402 INFO spark.SparkContext: Created broadcast 3 from showString at NativeMethodAccessorImpl.java:0
2023-12-17 09:39:23,407 INFO execution.FileSourceScanExec: Pla

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|              1|         2.10|         1|                 N|         142|          43|           2|          8|    3|    0.5|         0|           0|                  0.3

2023-12-17 09:39:59,983 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 1.0 (TID 1) in 36544 ms on 172.22.0.5 (executor 0) (1/1)
2023-12-17 09:39:59,983 INFO scheduler.TaskSchedulerImpl: Removed TaskSet 1.0, whose tasks have all completed, from pool 
2023-12-17 09:39:59,985 INFO scheduler.DAGScheduler: ResultStage 1 (showString at NativeMethodAccessorImpl.java:0) finished in 36.563 s
2023-12-17 09:39:59,985 INFO scheduler.DAGScheduler: Job 1 is finished. Cancelling potential speculative or zombie tasks for this job
2023-12-17 09:39:59,985 INFO scheduler.TaskSchedulerImpl: Killing all running tasks in stage 1: Stage finished
2023-12-17 09:39:59,986 INFO scheduler.DAGScheduler: Job 1 finished: showString at NativeMethodAccessorImpl.java:0, took 36.568292 s
2023-12-17 09:40:00,056 INFO codegen.CodeGenerator: Code generated in 54.085607 ms
2023-12-17 09:40:00,078 INFO storage.BlockManagerInfo: Removed broadcast_1_piece0 on a31eab60218a:43207 in memory (size: 5.9 KiB, free: 434.2 

In [5]:
df.write.parquet(f"wasbs://{os.environ['AZURE_STORAGE_CONTAINER']}@{os.environ['AZURE_STORAGE_ACCOUNT']}.blob.core.windows.net/yellow_tripdata/2021/01/", mode='overwrite')

2023-12-17 09:40:21,528 INFO datasources.FileSourceStrategy: Pushed Filters: 
2023-12-17 09:40:21,529 INFO datasources.FileSourceStrategy: Post-Scan Filters: 
2023-12-17 09:40:21,529 INFO datasources.FileSourceStrategy: Output Data Schema: struct<VendorID: string, tpep_pickup_datetime: string, tpep_dropoff_datetime: string, passenger_count: string, trip_distance: string ... 16 more fields>
2023-12-17 09:40:23,080 INFO parquet.ParquetFileFormat: Using default output committer for Parquet: org.apache.parquet.hadoop.ParquetOutputCommitter
2023-12-17 09:40:23,101 INFO output.FileOutputCommitter: File Output Committer Algorithm version is 1
2023-12-17 09:40:23,101 INFO output.FileOutputCommitter: FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
2023-12-17 09:40:23,102 INFO datasources.SQLHadoopMapReduceCommitProtocol: Using user defined output committer class org.apache.parquet.hadoop.ParquetOutputCommitter
2023-12-17 09:40:23,

KeyboardInterrupt: 

In [7]:
df_parquet = spark.read.parquet(f"wasbs://{os.environ['AZURE_STORAGE_CONTAINER']}@{os.environ['AZURE_STORAGE_ACCOUNT']}.blob.core.windows.net/yellow_tripdata/2021/01/")

2023-12-17 09:27:59,868 INFO datasources.InMemoryFileIndex: It took 554 ms to list leaf files for 1 paths.
2023-12-17 09:27:59,935 INFO spark.SparkContext: Starting job: parquet at NativeMethodAccessorImpl.java:0
2023-12-17 09:27:59,936 INFO scheduler.DAGScheduler: Got job 3 (parquet at NativeMethodAccessorImpl.java:0) with 1 output partitions
2023-12-17 09:27:59,936 INFO scheduler.DAGScheduler: Final stage: ResultStage 3 (parquet at NativeMethodAccessorImpl.java:0)
2023-12-17 09:27:59,936 INFO scheduler.DAGScheduler: Parents of final stage: List()
2023-12-17 09:27:59,937 INFO scheduler.DAGScheduler: Missing parents: List()
2023-12-17 09:27:59,938 INFO scheduler.DAGScheduler: Submitting ResultStage 3 (MapPartitionsRDD[16] at parquet at NativeMethodAccessorImpl.java:0), which has no missing parents
2023-12-17 09:27:59,957 INFO memory.MemoryStore: Block broadcast_7 stored as values in memory (estimated size 153.3 KiB, free 433.9 MiB)
2023-12-17 09:27:59,967 INFO memory.MemoryStore: Block

In [8]:
df_parquet.show()

2023-12-17 09:28:02,386 INFO datasources.FileSourceStrategy: Pushed Filters: 
2023-12-17 09:28:02,386 INFO datasources.FileSourceStrategy: Post-Scan Filters: 
2023-12-17 09:28:02,386 INFO datasources.FileSourceStrategy: Output Data Schema: struct<VendorID: string, tpep_pickup_datetime: string, tpep_dropoff_datetime: string, passenger_count: string, trip_distance: string ... 16 more fields>
2023-12-17 09:28:02,459 INFO codegen.CodeGenerator: Code generated in 40.494551 ms
2023-12-17 09:28:02,469 INFO memory.MemoryStore: Block broadcast_8 stored as values in memory (estimated size 307.8 KiB, free 433.9 MiB)
2023-12-17 09:28:02,480 INFO memory.MemoryStore: Block broadcast_8_piece0 stored as bytes in memory (estimated size 54.3 KiB, free 433.8 MiB)
2023-12-17 09:28:02,482 INFO storage.BlockManagerInfo: Added broadcast_8_piece0 in memory on a31eab60218a:42037 (size: 54.3 KiB, free: 434.3 MiB)
2023-12-17 09:28:02,483 INFO spark.SparkContext: Created broadcast 8 from showString at NativeMetho

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|              1|         2.10|         1|                 N|         142|          43|           2|          8|    3|    0.5|         0|           0|                  0.3

2023-12-17 09:28:10,555 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 4.0 (TID 5) in 8021 ms on 172.22.0.6 (executor 1) (1/1)
2023-12-17 09:28:10,555 INFO scheduler.TaskSchedulerImpl: Removed TaskSet 4.0, whose tasks have all completed, from pool 
2023-12-17 09:28:10,556 INFO scheduler.DAGScheduler: ResultStage 4 (showString at NativeMethodAccessorImpl.java:0) finished in 8.050 s
2023-12-17 09:28:10,556 INFO scheduler.DAGScheduler: Job 4 is finished. Cancelling potential speculative or zombie tasks for this job
2023-12-17 09:28:10,556 INFO scheduler.TaskSchedulerImpl: Killing all running tasks in stage 4: Stage finished
2023-12-17 09:28:10,557 INFO scheduler.DAGScheduler: Job 4 finished: showString at NativeMethodAccessorImpl.java:0, took 8.055501 s


In [9]:
spark.stop()

2023-12-17 09:28:10,604 INFO server.AbstractConnector: Stopped Spark@411e682a{HTTP/1.1, (http/1.1)}{0.0.0.0:4040}
2023-12-17 09:28:10,609 INFO ui.SparkUI: Stopped Spark web UI at http://a31eab60218a:4040
2023-12-17 09:28:10,615 INFO cluster.StandaloneSchedulerBackend: Shutting down all executors
2023-12-17 09:28:10,616 INFO cluster.CoarseGrainedSchedulerBackend$DriverEndpoint: Asking each executor to shut down
2023-12-17 09:28:11,040 INFO spark.MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
2023-12-17 09:28:11,056 INFO memory.MemoryStore: MemoryStore cleared
2023-12-17 09:28:11,057 INFO storage.BlockManager: BlockManager stopped
2023-12-17 09:28:11,060 INFO storage.BlockManagerMaster: BlockManagerMaster stopped
2023-12-17 09:28:11,064 INFO scheduler.OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
2023-12-17 09:28:11,078 INFO spark.SparkContext: Successfully stopped SparkContext
